# Clausius Rankine Cycle


```{figure} /figures/rankine_cycle_dissipative.svg
---
name: rankine-cycle-dissipative-flowsheet
---
Topology of the clausius rankine cycle.
```


## Introduction

The Clausius-Rankine cycle serves as a comparative process for the steam power plant in its simplest constellation with steam turbine, condenser, feed water pump and steam generator. The steam power process is based on the heat-power process taking place in a steam engine. In contrast to the first steam engines, the working medium is kept in a closed circuit. This means that the process is also a clockwise rotating cycle.

The live steam provided in the steam generator enters the turbine at high pressure and temperature, where it is expanded to a lower pressure level. The turbine releases power that can be used to drive a generator. The exhaust steam leaving the turbine is completely condensed out and leaves the condenser as saturated water. After that the feed water pump has to bring this condensate to a pressure level so that the steam generator can be supplied with feed water and the cycle can start again.

The heat supplied to the process comes from the steam generator and is to be regarded as fuel. In contrast, the power output of the turbine, differentiated by the power  input of the feedwater pump, is to be defined as the product of the process. The complete condensation of the refrigerant are to be considered as exergetic losses in the heat-power process.


## Excercise 1 

Build the clausius rankine cycle in two variants:

1. A cycle with a dissipative condenser (system boundaries are around the hot side of the heat exchanger)
2. A condenser with the cold side, e.g. a river water cooling cycle, modelled (system boundaries are in and out flowing water)

Make sure the cycles are the same by checking their respective thermal efficiencies. Perform and compare the exergy analysis of the two variants. Interprete the results and the implication on the condensers efficiency. 

### Proposed solution 1.1

**First step:** Build clausius rankine cycle with condenser using `HeatExchangerSimple` component

In [ ]:
from tespy.networks import Network
from tespy.connections import Connection, Bus
from tespy.components import CycleCloser, Pump, HeatExchangerSimple, Turbine

# Network
nw = Network(fluids=['water'])
nw.set_attr(T_unit='C', p_unit='bar', h_unit='kJ / kg')

# Components
cycle_closer = CycleCloser('Cycle Closer')
turbine = Turbine('Steam Turbine')
condenser = HeatExchangerSimple('Condenser')
fw_pump = Pump('Feed Water Pump')
steam_generator = HeatExchangerSimple('Steam Generator')

# Connections
steamgen2turb = Connection(steam_generator, 'out1', cycle_closer, 'in1', label='0')
cc2turb = Connection(cycle_closer, 'out1', turbine, 'in1', label='1')
turb2cond = Connection(turbine, 'out1', condenser, 'in1', label='2')
cond2pump = Connection(condenser, 'out1', fw_pump, 'in1', label='3')
pump2steamgen = Connection(fw_pump, 'out1', steam_generator, 'in1', label='4')

nw.add_conns(steamgen2turb, cc2turb, turb2cond, cond2pump, pump2steamgen)

# Component Parametrization
turbine.set_attr(eta_s=0.9)
condenser.set_attr(pr=1)
fw_pump.set_attr(eta_s=0.75)
steam_generator.set_attr(pr=0.9)

# Connection Parametrization
cc2turb.set_attr(T=600, p=150, m=10, fluid={'water': 1})
turb2cond.set_attr(p=0.1)
cond2pump.set_attr(x=0)

# Busses
heat_in = Bus('Heat Input')
heat_in.add_comps({'comp': steam_generator, 'base': 'bus'})

power_out = Bus('Power Output')
power_out.add_comps(
    {'comp': turbine, 'char': 0.96, 'base': 'component'},
    {'comp': fw_pump, 'char': 0.96, 'base': 'bus'}
    )

heat_out = Bus('Heat Output')
heat_out.add_comps({'comp': condenser, 'base': 'component'})

nw.add_busses(heat_in, power_out, heat_out)

**Second step:** Solve model with variant 1 and perform exergy analysis

In [ ]:
from tespy.tools import ExergyAnalysis

# Solve model
nw.set_attr(iterinfo=False)
nw.solve(mode='design')
eta_th = abs(power_out.P.val)/heat_in.P.val
print(f'eta_th = {eta_th:.3f}')

ean_var1 = ExergyAnalysis(network=nw, E_F=[heat_in], E_P=[power_out], E_L=[heat_out])
ean_var1.analyse(pamb=1.013, Tamb=20)
ean_var1.print_results(groups=False, connections=False, aggregation=False)

**Third step:** Replace `HeatExchangerSimple` with `Condenser` component

In [ ]:
from tespy.components import Condenser, Source, Sink
    
# Remove old connections
nw.del_conns(turb2cond, cond2pump)

# Replace condenser
condenser = Condenser('Condenser')
cw_source = Source('Cooling Water Source')
cw_sink = Sink('Cooling Water Sink')

# Reconnect condenser
turb2cond = Connection(turbine, 'out1', condenser, 'in1', label='2')
cond2pump = Connection(condenser, 'out1', fw_pump, 'in1', label='3')
cw_source2cond = Connection(cw_source, 'out1', condenser, 'in2', label='11')
cond2cw_sink = Connection(condenser, 'out2', cw_sink, 'in1', label='12')

nw.add_conns(turb2cond, cond2pump, cw_source2cond, cond2cw_sink)

# Parametrize condenser and cooling cycle
turb2cond.set_attr(p=0.1)
condenser.set_attr(pr1=1, pr2=1)
cw_source2cond.set_attr(T=20, p=1.2, fluid={'water': 1})
cond2cw_sink.set_attr(T=30)

# Reset heat output bus
nw.del_busses(heat_out)

heat_out = Bus('Heat Output')
heat_out.add_comps(
    {'comp': cw_source, 'base': 'bus'},
    {'comp': cw_sink, 'base': 'component'}
    )

nw.add_busses(heat_out)

**Forth step:** Solve model with variant 2 and perform exergy analysis

In [ ]:
# Solve model
nw.set_attr(iterinfo=False)
nw.solve(mode='design')
eta_th = abs(power_out.P.val)/heat_in.P.val
print(f'eta_th = {eta_th:.3f}')

ean_var2 = ExergyAnalysis(network=nw, E_F=[heat_in], E_P=[power_out], E_L=[heat_out])
ean_var2.analyse(pamb=1.013, Tamb=20)
ean_var2.print_results(groups=False, connections=False, aggregation=False)

```{admonition} Explanation
:class: dropdown

First variation makes Condenser look very inefficient, because all exergy is destroyed. Second variation highlights real losses of the condenser and dissipates heat through cooling water.
```


## Excercise 2

```{attention}

From this point onwards the second variant of the clausius rankine cycle containg the implementation of the condensers cooling cycle is used.
```

1. Analyze the influence of key parameters on the exergy analysis, such as live steam pressure and temperature.
2. What impact does the ambient state have on the results of the exergy analysis.
3. Implement the following two topological improvements:
    - Regenerative feed water preheating (see {numref}`rankine-cycle-preheating-flowsheet`)
    - Intermediate superheating (see Figure XX)
4. Explain the improvement of the cycles efficiency with the exergy analysis by investigating the exergy destruction of each component

```{figure} /figures/rankine_cycle_preheating.svg
---
name: rankine-cycle-preheating-flowsheet
---
Topology of the clausius rankine cycle with regenerative feed water pre heating.
```